# Footprint Integration with Costmaps

This notebook teaches you how to add a **footprint module** to your autonomous car stack and integrate it with costmaps for accurate collision avoidance.

## What You'll Learn

1. **What is a footprint?** - Understanding vehicle shape representation
2. **Creating footprints** - Rectangular and circular footprints
3. **Integrating with costmaps** - Using footprint-based inflation instead of arbitrary radius
4. **Benefits** - Why footprint-based inflation is better

## Why Footprints?

Traditionally, costmaps use a **fixed inflation radius** around obstacles. This works, but:
- The radius is arbitrary (often too large or too small)
- Doesn't account for actual vehicle shape
- Can be inefficient (over-inflating) or unsafe (under-inflating)

**Footprint-based inflation** uses the actual vehicle shape to calculate the minimum safe distance, ensuring:
- The entire vehicle footprint avoids obstacles
- More accurate and efficient path planning
- Better safety margins

## Prerequisites

- Understanding of costmaps (see [Building a Costmap](../costmaps/learning_build_costmap.ipynb))
- Basic geometry knowledge


In [ ]:
import sys

sys.path.insert(0, '../../../src')

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    BaseFootprint,
    CarState,
    GridCostmap,
    GridMap,
    RectangularFootprint,
)

## Step 1: Understanding Footprints

A **footprint** represents the shape of your vehicle in 2D space. It answers: "What area does my vehicle occupy?"

### Key Concepts

- **Vertices**: The corners/points that define the footprint shape
- **Bounding Radius**: Distance from center to farthest point (for efficient collision checking)
- **Inflation Radius**: Bounding radius + safety padding (used for costmap inflation)

### Example: Car Footprint

A typical car might be:
- **Length**: 4.5 meters (front to back)
- **Width**: 1.8 meters (left to right)
- **Shape**: Rectangle (approximation)

The footprint ensures that when we plan a path, the **entire rectangle** avoids obstacles, not just a point.

In [ ]:
# Create a rectangular footprint for a typical car
car_footprint = RectangularFootprint(
    length=4.5,  # 4.5 meters long
    width=1.8,  # 1.8 meters wide
    name="car_footprint"
)

print(f"Footprint: {car_footprint.name}")
print(f"  Length: {car_footprint.length}m")
print(f"  Width: {car_footprint.width}m")
print(f"  Bounding radius: {car_footprint.get_bounding_radius():.2f}m")
print(f"  Inflation radius (no padding): {car_footprint.get_inflation_radius():.2f}m")
print(f"  Inflation radius (0.5m padding): {car_footprint.get_inflation_radius(padding=0.5):.2f}m")

## Step 2: Visualizing Footprints

Let's visualize how footprints work at different positions and orientations.

In [ ]:
def visualize_footprint(footprint: BaseFootprint, position: np.ndarray, heading: float, ax=None):
    """Visualize footprint at a given position and heading."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))

    # Get footprint vertices
    vertices = footprint.get_vertices(position, heading)

    # Draw footprint as polygon
    if isinstance(footprint, RectangularFootprint):
        polygon = patches.Polygon(vertices, closed=True, color='blue', alpha=0.3, label='Footprint')
    else:
        polygon = patches.Circle(position, footprint.radius, color='blue', alpha=0.3, label='Footprint')
    ax.add_patch(polygon)

    # Draw vehicle position
    ax.plot(position[0], position[1], 'ro', markersize=10, label='Vehicle Center')

    # Draw heading arrow
    arrow_length = footprint.get_bounding_radius() * 0.8
    dx = arrow_length * np.cos(heading)
    dy = arrow_length * np.sin(heading)
    ax.arrow(position[0], position[1], dx, dy, head_width=0.3, head_length=0.3,
             fc='red', ec='red', label='Heading')

    # Draw bounding circle
    circle = patches.Circle(position, footprint.get_bounding_radius(),
                           fill=False, linestyle='--', color='gray', label='Bounding Radius')
    ax.add_patch(circle)

    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_title(f'Footprint at position [{position[0]:.1f}, {position[1]:.1f}], heading {np.degrees(heading):.1f}°')

    return ax

# Visualize at different positions/orientations
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

positions = [np.array([0, 0]), np.array([5, 2]), np.array([-3, 4])]
headings = [0.0, np.pi/4, np.pi/2]

for i, (pos, heading) in enumerate(zip(positions, headings)):
    visualize_footprint(car_footprint, pos, heading, ax=axes[i])

plt.tight_layout()
plt.show()

## Step 3: Footprint-Based Costmap Inflation

Now let's see how to use footprints with costmaps. Instead of using an arbitrary inflation radius, we calculate it from the footprint.

In [ ]:
# Create a grid map with obstacles
grid_map = GridMap.create_random_map(
    width=30.0,
    height=30.0,
    num_obstacles=10,
    obstacle_size=1.5,
    seed=42
)

# Create footprint
footprint = RectangularFootprint(length=4.5, width=1.8)

# Create costmap WITH footprint (inflation radius calculated automatically)
costmap_with_footprint = GridCostmap(
    width=30.0,
    height=30.0,
    resolution=0.5,
    footprint=footprint,  # Pass footprint here
    footprint_padding=0.5,  # 0.5m safety padding
    frame="global"
)

# Create costmap WITHOUT footprint (uses arbitrary radius)
costmap_without_footprint = GridCostmap(
    width=30.0,
    height=30.0,
    resolution=0.5,
    inflation_radius=2.5,  # Arbitrary radius
    frame="global"
)

# Update both costmaps with same obstacles
car_state = CarState(x=0.0, y=0.0, heading=0.0)
all_obstacles = np.vstack([grid_map.obstacles, grid_map.get_boundary_obstacles()])

costmap_with_footprint.update(static_obstacles=all_obstacles, car_state=car_state)
costmap_without_footprint.update(static_obstacles=all_obstacles, car_state=car_state)

print(f"Footprint-based inflation radius: {costmap_with_footprint.inflation_radius:.2f}m")
print(f"Arbitrary inflation radius: {costmap_without_footprint.inflation_radius:.2f}m")
print(f"\nFootprint bounding radius: {footprint.get_bounding_radius():.2f}m")
print(f"Footprint inflation (with padding): {footprint.get_inflation_radius(padding=0.5):.2f}m")

## Step 4: Comparing Footprint vs Arbitrary Inflation

Let's visualize the difference between footprint-based and arbitrary inflation.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Plot costmap with footprint
ax = axes[0]
grid_map.visualize(ax=ax, frame="global")
costmap_with_footprint.visualize(ax=ax, car_state=car_state, frame="global", alpha=0.4)
ax.set_title(f'Footprint-Based Inflation (radius={costmap_with_footprint.inflation_radius:.2f}m)')
ax.set_xlim(-15, 15)
ax.set_ylim(-15, 15)

# Plot costmap without footprint
ax = axes[1]
grid_map.visualize(ax=ax, frame="global")
costmap_without_footprint.visualize(ax=ax, car_state=car_state, frame="global", alpha=0.4)
ax.set_title(f'Arbitrary Inflation (radius={costmap_without_footprint.inflation_radius:.2f}m)')
ax.set_xlim(-15, 15)
ax.set_ylim(-15, 15)

plt.tight_layout()
plt.show()

print("Notice how footprint-based inflation is more accurate!")

## Step 5: Building Your Own Footprint

You can create custom footprints by extending `BaseFootprint`. Let's build a custom footprint!

In [ ]:
class CustomFootprint(BaseFootprint):
    """
    Custom footprint example: L-shaped vehicle (like a forklift).

    TODO: Fill in the implementation!
    """

    def __init__(self, front_length: float, rear_length: float, width: float, name: str = "custom_footprint"):
        """
        Initialize L-shaped footprint.

        TODO:
        1. Call super().__init__(name=name)
        2. Store front_length, rear_length, width
        3. Calculate bounding_radius (distance from center to farthest corner)
        """
        super().__init__(name=name)
        self.front_length = front_length
        self.rear_length = rear_length
        self.width = width

        # TODO: Calculate bounding radius
        # Hint: Find the farthest corner from center
        max_dist = max(
            np.sqrt((front_length/2)**2 + (width/2)**2),  # Front corner
            np.sqrt((rear_length/2)**2 + (width/2)**2)    # Rear corner
        )
        self._bounding_radius = max_dist

    def get_vertices(self, position: np.ndarray, heading: float) -> np.ndarray:
        """
        Get L-shaped vertices.

        TODO:
        1. Define vertices in vehicle frame (centered at origin)
        2. Rotate by heading
        3. Translate to position
        """
        # TODO: Define L-shape vertices in vehicle frame
        # Front part: rectangle from -rear_length/2 to front_length/2, width/2 to -width/2
        # But we'll make it simpler: just a rectangle for now
        # You can make it more complex!

        half_front = self.front_length / 2
        half_rear = self.rear_length / 2
        half_width = self.width / 2

        # Simple rectangle (you can make this L-shaped!)
        vertices_vehicle = np.array([
            [half_front, half_width],
            [-half_rear, half_width],
            [-half_rear, -half_width],
            [half_front, -half_width],
        ])

        # TODO: Rotate by heading
        cos_h = np.cos(heading)
        sin_h = np.sin(heading)
        rotation_matrix = np.array([
            [cos_h, -sin_h],
            [sin_h, cos_h]
        ])
        vertices_rotated = (rotation_matrix @ vertices_vehicle.T).T

        # TODO: Translate to position
        vertices = vertices_rotated + position

        return vertices

    def get_bounding_radius(self) -> float:
        """Get bounding radius."""
        return self._bounding_radius

    def contains_point(self, point: np.ndarray, position: np.ndarray, heading: float) -> bool:
        """
        Check if point is inside footprint.

        TODO:
        1. Transform point to vehicle frame
        2. Check if within bounds
        """
        # TODO: Transform point to vehicle frame
        point_relative = point - position

        # Rotate by -heading
        cos_h = np.cos(-heading)
        sin_h = np.sin(-heading)
        rotation_matrix = np.array([
            [cos_h, -sin_h],
            [sin_h, cos_h]
        ])
        point_vehicle = rotation_matrix @ point_relative

        # TODO: Check bounds (simple rectangle check)
        half_front = self.front_length / 2
        half_rear = self.rear_length / 2
        half_width = self.width / 2

        return (-half_rear <= point_vehicle[0] <= half_front and
                -half_width <= point_vehicle[1] <= half_width)

# Test custom footprint
custom_footprint = CustomFootprint(front_length=3.0, rear_length=2.0, width=1.5)
print(f"Custom footprint bounding radius: {custom_footprint.get_bounding_radius():.2f}m")

# Visualize
fig, ax = plt.subplots(figsize=(8, 8))
visualize_footprint(custom_footprint, np.array([0, 0]), np.pi/4, ax=ax)
plt.show()

## Step 6: Using Footprints in Simulation

Now let's see how to integrate footprints into a full simulation.

In [ ]:
from simple_autonomous_car import Car, GoalPlanner, PurePursuitController

# Create components
grid_map = GridMap.create_random_map(width=40.0, height=40.0, num_obstacles=15, seed=42)

# Create footprint
footprint = RectangularFootprint(length=4.5, width=1.8)

# Create costmap with footprint
costmap = GridCostmap(
    width=40.0,
    height=40.0,
    resolution=0.5,
    footprint=footprint,  # Use footprint!
    footprint_padding=0.3,  # 30cm safety margin
    frame="ego"
)

# Create car
car = Car(initial_state=CarState(x=-15.0, y=-15.0, heading=0.0, velocity=3.0))

# Create planner
planner = GoalPlanner(grid_map=grid_map, resolution=0.5)

# Create controller
controller = PurePursuitController(
    lookahead_distance=6.0,
    target_velocity=3.0
)

# Goal
goal = np.array([15.0, 15.0])

# Simulate a few steps
dt = 0.1
trajectory = [car.state.position().copy()]

for step in range(50):
    # Update costmap with obstacles
    all_obstacles = np.vstack([grid_map.obstacles, grid_map.get_boundary_obstacles()])
    costmap.update(static_obstacles=all_obstacles, car_state=car.state)

    # Plan
    plan = planner.plan(car.state, costmap=costmap, goal=goal)

    # Control
    control = controller.compute_control(
        car.state,
        plan=plan,
        costmap=costmap,
        goal=goal,
        goal_tolerance=2.0
    )

    # Update car
    car.update(dt, acceleration=control["acceleration"], steering_rate=control["steering_rate"])
    trajectory.append(car.state.position().copy())

    if np.linalg.norm(car.state.position() - goal) < 2.0:
        print(f"Goal reached at step {step}!")
        break

# Visualize
fig, ax = plt.subplots(figsize=(12, 12))

# Plot map and obstacles
grid_map.visualize(ax=ax, frame="global")

# Plot costmap
costmap.visualize(ax=ax, car_state=car.state, frame="global", alpha=0.3)

# Plot trajectory
trajectory = np.array(trajectory)
ax.plot(trajectory[:, 0], trajectory[:, 1], 'g-', linewidth=2, label='Trajectory')

# Plot final footprint
final_vertices = footprint.get_vertices(car.state.position(), car.state.heading)
final_polygon = patches.Polygon(final_vertices, closed=True, color='blue', alpha=0.5, label='Final Footprint')
ax.add_patch(final_polygon)

# Plot goal
ax.plot(goal[0], goal[1], 'ro', markersize=15, label='Goal', markeredgecolor='black', markeredgewidth=2)

ax.set_xlim(-20, 20)
ax.set_ylim(-20, 20)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_title('Simulation with Footprint-Based Costmap')
plt.show()

print(f"\nCostmap inflation radius (from footprint): {costmap.inflation_radius:.2f}m")
print(f"Footprint bounding radius: {footprint.get_bounding_radius():.2f}m")
print("Safety padding: 0.3m")

## Step 7: Advanced: Footprint Collision Checking

Footprints can also be used for direct collision checking, not just costmap inflation.

In [ ]:
# Check if a path is collision-free using footprint
def check_path_collision(path: np.ndarray, footprint: BaseFootprint, obstacles: np.ndarray) -> Tuple[bool, int]:
    """
    Check if a path collides with obstacles using footprint.

    Returns:
        (is_collision_free, first_collision_idx)
    """
    for i, waypoint in enumerate(path):
        # Estimate heading (direction to next waypoint)
        if i < len(path) - 1:
            direction = path[i + 1] - waypoint
            heading = np.arctan2(direction[1], direction[0])
        else:
            # Last waypoint, use previous heading
            if i > 0:
                direction = waypoint - path[i - 1]
                heading = np.arctan2(direction[1], direction[0])
            else:
                heading = 0.0

        # Check collision at this waypoint
        if footprint.check_collision(obstacles, waypoint, heading):
            return False, i

    return True, len(path)

# Test collision checking
test_path = np.array([
    [-15, -15],
    [-10, -10],
    [-5, -5],
    [0, 0],
    [5, 5],
    [10, 10],
    [15, 15]
])

all_obstacles = np.vstack([grid_map.obstacles, grid_map.get_boundary_obstacles()])
is_safe, collision_idx = check_path_collision(test_path, footprint, all_obstacles)

print("Path collision check:")
print(f"  Is safe: {is_safe}")
if not is_safe:
    print(f"  First collision at waypoint {collision_idx}: {test_path[collision_idx]}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 10))
grid_map.visualize(ax=ax, frame="global")

# Plot path
ax.plot(test_path[:, 0], test_path[:, 1], 'g-o', linewidth=2, markersize=6, label='Path')

# Plot footprints along path
for i, waypoint in enumerate(test_path[::3]):  # Every 3rd waypoint
    if i < len(test_path) - 1:
        direction = test_path[min(i*3+1, len(test_path)-1)] - waypoint
        heading = np.arctan2(direction[1], direction[0])
    else:
        heading = 0.0

    vertices = footprint.get_vertices(waypoint, heading)
    polygon = patches.Polygon(vertices, closed=True, color='blue', alpha=0.2)
    ax.add_patch(polygon)

ax.set_xlim(-20, 20)
ax.set_ylim(-20, 20)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_title('Path Collision Checking with Footprints')
plt.show()

## Summary

You've learned:

1. **What footprints are**: Vehicle shape representation for accurate collision avoidance
2. **Creating footprints**: `RectangularFootprint` and `CircularFootprint` classes
3. **Footprint-based inflation**: Using footprint bounding radius + padding instead of arbitrary radius
4. **Integration with costmaps**: Pass `footprint` parameter to `GridCostmap`
5. **Collision checking**: Using footprints to check if paths are safe

### Key Benefits

- **More accurate**: Inflation radius matches actual vehicle size
- **More efficient**: No over-inflation (wasteful) or under-inflation (unsafe)
- **Flexible**: Easy to change vehicle size or create custom shapes
- **Safe**: Ensures entire vehicle footprint avoids obstacles

### Next Steps

- Experiment with different footprint sizes
- Try creating custom footprint shapes (L-shaped, trapezoidal, etc.)
- Integrate footprints into your planners for better path validation
- Use footprints in controllers to check if control commands are safe